In [1]:
import requests, pickle, time
from urllib.parse import urljoin
from multiprocessing.pool import ThreadPool, Pool
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import threading

import pandas as pd

# Load Country Standardizers

In [2]:
# iso = pd.read_csv('country_codes.csv')
# iso = iso[['official_name_en', 'ISO3166-1-Alpha-3']]
# iso = iso.rename(columns={'ISO3166-1-Alpha-3': 'iso'})
# iso = iso[~pd.isna(iso['official_name_en'])]
# iso = dict(zip(list(iso['official_name_en']), list(iso['iso'])))

with open('iso.pk', 'rb') as f:
    iso = pickle.load(f)

In [3]:
def standardize_iso(x, inp=True):
    if x in iso: return iso[x]
    if pd.isna(x): return x
    else:
        if inp:
            y = input('what iso should ' + x + ' map to? ')
            iso[x] = y
            return y

In [4]:
# from the location dropdown here: http://www.hoovers.com/company-information/company-search.html

x = '<option value="76">United States (17760172)</option><option value="14">Canada (1800305)</option><option value="128">Afghanistan (2076)</option><option value="129">Albania (2538)</option><option value="1">Algeria (144909)</option><option value="130">American Samoa (191)</option><option value="131">Andorra (490)</option><option value="132">Angola (630)</option><option value="133">Anguilla (93)</option><option value="2">Antigua and Barbuda (114)</option><option value="3">Argentina (1859746)</option><option value="134">Armenia (712)</option><option value="135">Aruba (354)</option><option value="4">Australia (601943)</option><option value="5">Austria (551332)</option><option value="111">Azerbaijan (803)</option><option value="6">Bahamas (722)</option><option value="7">Bahrain (43531)</option><option value="137">Bangladesh (3414)</option><option value="97">Barbados (517)</option><option value="138">Belarus (188879)</option><option value="8">Belgium (1687423)</option><option value="9">Belize (1064)</option><option value="139">Benin (475)</option><option value="10">Bermuda (1424)</option><option value="140">Bhutan (100)</option><option value="11">Bolivia (1367)</option><option value="119">Bosnia-Herzegovina (33209)</option><option value="99">Botswana (3505)</option><option value="12">Brazil (16684304)</option><option value="13">British Virgin Islands (3868)</option><option value="141">Brunei (134)</option><option value="83">Bulgaria (500431)</option><option value="142">Burkina Faso (238)</option><option value="143">Burundi (84)</option><option value="144">Cambodia (733)</option><option value="145">Cameroon (802)</option><option value="146">Cape Verde (83)</option><option value="15">Cayman Islands (3272)</option><option value="148">Central African Republic (40)</option><option value="149">Chad (119)</option><option value="17">Chile (783123)</option><option value="18">China (18609072)</option><option value="19">Colombia (1967071)</option><option value="151">Comoros (30)</option><option value="205">Congo (Brazzaville) (187)</option><option value="153">Cook Islands (53)</option><option value="20">Costa Rica (3733)</option><option value="84">Croatia (125377)</option><option value="52">Curaçao (13)</option><option value="85">Cyprus (26000)</option><option value="21">Czech Republic (1332294)</option><option value="154">Côte d\'Ivoire (811)</option><option value="152">Democratic Republic of the Congo (869)</option><option value="22">Denmark (476684)</option><option value="156">Djibouti (123)</option><option value="157">Dominica (64)</option><option value="23">Dominican Republic (118662)</option><option value="158">East Timor (1)</option><option value="24">Ecuador (97829)</option><option value="100">Egypt (3444)</option><option value="25">El Salvador (1549)</option><option value="245">England (2415048)</option><option value="159">Equatorial Guinea (39)</option><option value="160">Eritrea (13)</option><option value="86">Estonia (119270)</option><option value="124">Ethiopia (676)</option><option value="162">Falkland Islands (15)</option><option value="161">Faroe Islands (1080)</option><option value="123">Fiji (431)</option><option value="26">Finland (575694)</option><option value="27">France (3494458)</option><option value="117">French Guiana (12353)</option><option value="163">French Polynesia (332)</option><option value="164">Gabon (196)</option><option value="165">Gambia (147)</option><option value="167">Georgia (1254)</option><option value="28">Germany (3766265)</option><option value="29">Ghana (1610)</option><option value="30">Gibraltar (660)</option><option value="31">Greece (94032)</option><option value="168">Greenland (8161)</option><option value="169">Grenada (53)</option><option value="170">Guadeloupe (47036)</option><option value="171">Guam (1415)</option><option value="32">Guatemala (140118)</option><option value="172">Guinea (158)</option><option value="173">Guinea-Bissau (33)</option><option value="174">Guyana (236)</option><option value="126">Haiti (590)</option><option value="33">Honduras (1386)</option><option value="34">Hong Kong (237378)</option><option value="35">Hungary (476286)</option><option value="87">Iceland (11512)</option><option value="36">India (4730270)</option><option value="38">Indonesia (50983)</option><option value="81">Iran (14233)</option><option value="175">Iraq (3355)</option><option value="39">Ireland (129484)</option><option value="40">Israel (199897)</option><option value="41">Italy (6194919)</option><option value="42">Jamaica (814)</option><option value="43">Japan (4708718)</option><option value="44">Jordan (3312)</option><option value="176">Kazakhstan (434362)</option><option value="101">Kenya (2832)</option><option value="177">Kiribati (7)</option><option value="392">Kosovo (1620)</option><option value="102">Kuwait (7206)</option><option value="178">Kyrgyzstan (89639)</option><option value="179">Laos (11)</option><option value="88">Latvia (122345)</option><option value="113">Lebanon (9803)</option><option value="180">Lesotho (428)</option><option value="103">Liberia (406)</option><option value="46">Libya (289)</option><option value="47">Liechtenstein (9875)</option><option value="89">Lithuania (69760)</option><option value="48">Luxembourg (145820)</option><option value="181">Macao (910)</option><option value="90">Macedonia (3823)</option><option value="182">Madagascar (281)</option><option value="104">Malawi (66)</option><option value="49">Malaysia (10299)</option><option value="183">Maldives (136)</option><option value="184">Mali (214)</option><option value="91">Malta (3079)</option><option value="185">Marshall Islands (166)</option><option value="186">Martinique (37970)</option><option value="187">Mauritania (137)</option><option value="115">Mauritius (13249)</option><option value="50">Mexico (219761)</option><option value="189">Micronesia (156)</option><option value="191">Moldova (101357)</option><option value="51">Monaco (310)</option><option value="250">Montenegro (33543)</option><option value="193">Montserrat (11)</option><option value="105">Morocco (477946)</option><option value="194">Mozambique (837)</option><option value="195">Myanmar (60)</option><option value="106">Namibia (3060)</option><option value="196">Nauru (5)</option><option value="197">Nepal (521)</option><option value="53">New Zealand (305314)</option><option value="54">Nicaragua (875)</option><option value="199">Niger (159)</option><option value="55">Nigeria (6249)</option><option value="252">Norfolk Island (6)</option><option value="246">Northern Ireland (55329)</option><option value="202">Northern Mariana Islands (271)</option><option value="56">Norway (843882)</option><option value="203">Oman (5636)</option><option value="107">Pakistan (2100)</option><option value="57">Panama (3667)</option><option value="58">Papua New Guinea (619)</option><option value="59">Paraguay (1285)</option><option value="60">Peru (709808)</option><option value="61">Philippines (46219)</option><option value="82">Poland (4358713)</option><option value="62">Portugal (608961)</option><option value="121">Qatar (5842)</option><option value="92">Romania (750475)</option><option value="63">Russia (2950310)</option><option value="206">Rwanda (452)</option><option value="118">Réunion (44430)</option><option value="207">Samoa (101)</option><option value="208">San Marino (3494)</option><option value="64">Saudi Arabia (3671)</option><option value="247">Scotland (193534)</option><option value="210">Senegal (783)</option><option value="96">Serbia (136704)</option><option value="211">Seychelles (252)</option><option value="212">Sierra Leone (235)</option><option value="65">Singapore (422433)</option><option value="271">Sint Maarten (Dutch Part) (3)</option><option value="93">Slovakia (561260)</option><option value="94">Slovenia (178210)</option><option value="213">Solomon Islands (61)</option><option value="214">Somalia (145)</option><option value="66">South Africa (253296)</option><option value="45">South Korea (2459815)</option><option value="273">South Sudan (92)</option><option value="67">Spain (2725654)</option><option value="108">Sri Lanka (1453)</option><option value="255">St. Helena (3)</option><option value="218">St. Kitts and Nevis (140)</option><option value="219">St. Lucia (135)</option><option value="221">St. Vincent and the Grenadines (158)</option><option value="222">Sudan (254)</option><option value="114">Suriname (242)</option><option value="223">Swaziland (1162)</option><option value="68">Sweden (1290234)</option><option value="69">Switzerland (674767)</option><option value="224">Syria (1794)</option><option value="209">São Tomé and Príncipe (18)</option><option value="70">Taiwan (455412)</option><option value="225">Tajikistan (209)</option><option value="226">Tanzania (1249)</option><option value="71">Thailand (548502)</option><option value="72">The Netherlands (668558)</option><option value="227">Togo (311)</option><option value="229">Tonga (54)</option><option value="98">Trinidad and Tobago (908)</option><option value="109">Tunisia (100581)</option><option value="73">Turkey (317443)</option><option value="763">Turkish Republic of Northern Cyprus (261)</option><option value="230">Turkmenistan (67)</option><option value="122">Turks and Caicos (66)</option><option value="231">Tuvalu (2)</option><option value="232">Uganda (1269)</option><option value="95">Ukraine (941926)</option><option value="75">United Arab Emirates (22431)</option><option value="74">United Kingdom (1)</option><option value="77">Uruguay (2327)</option><option value="237">Uzbekistan (374)</option><option value="238">Vanuatu (80)</option><option value="78">Venezuela (2974)</option><option value="125">Vietnam (635678)</option><option value="256">Wake Island (2)</option><option value="248">Wales (104733)</option><option value="243">Yemen (878)</option><option value="79">Zambia (683)</option><option value="110">Zimbabwe (418)</option>'
x = BeautifulSoup(x, 'lxml').find_all('option')
x = [standardize_iso(i.text.split(' (')[0]) for i in x]
x


['USA',
 'CAN',
 'AFG',
 'ALB',
 'DZA',
 'ASM',
 'AND',
 'AGO',
 'AIA',
 'ATG',
 'ARG',
 'ARM',
 'ABW',
 'AUS',
 'AUT',
 'AZE',
 'BHS',
 'BHR',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BMU',
 'BTN',
 'BOL',
 'BIH',
 'BWA',
 'BRA',
 'VGB',
 'BRN',
 'BGR',
 'BFA',
 'BDI',
 'KHM',
 'CMR',
 'CPV',
 'CYM',
 'CAF',
 'TCD',
 'CHL',
 'CHN',
 'COL',
 'COM',
 'COG',
 'COK',
 'CRI',
 'HRV',
 'CUW',
 'CYP',
 'CZE',
 'CIV',
 'COD',
 'DNK',
 'DJI',
 'DMA',
 'DOM',
 'TLS',
 'ECU',
 'EGY',
 'SLV',
 'GBR',
 'GNQ',
 'ERI',
 'EST',
 'ETH',
 'FLK',
 'FRO',
 'FJI',
 'FIN',
 'FRA',
 'GUF',
 'PYF',
 'GAB',
 'GMB',
 'GEO',
 'DEU',
 'GHA',
 'GIB',
 'GRC',
 'GRL',
 'GRD',
 'GLP',
 'GUM',
 'GTM',
 'GIN',
 'GNB',
 'GUY',
 'HTI',
 'HND',
 'HKG',
 'HUN',
 'ISL',
 'IND',
 'IDN',
 'IRN',
 'IRQ',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KIR',
 'RKS',
 'KWT',
 'KGZ',
 'LAO',
 'LVA',
 'LBN',
 'LSO',
 'LBR',
 'LBY',
 'LIE',
 'LTU',
 'LUX',
 'MAC',
 'MKD',
 'MDG',
 'MWI',
 'MYS',
 'MDV',


In [5]:
# import pickle
# with open('iso.pk', 'wb') as f:
#     pickle.dump(iso, f)

# Load CDP Data

In [6]:
cdp = pd.read_csv('cdp_filtered.csv')
cdp = cdp.drop(columns=['Unnamed: 0'])
cdp = cdp[~cdp.duplicated()]
cdp = cdp.reset_index(drop=True)

In [7]:
cdp['iso'] = cdp['country'].apply(standardize_iso)

what iso should Taiwan, Greater China map to? 


# Scraper

In [8]:
# Threading stuff

threadLocal = threading.local()

def get_driver():
    driver = getattr(threadLocal, 'driver', None)
    if driver is None:
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        driver = webdriver.Chrome(options=chromeOptions)
        driver.implicitly_wait(0)
        setattr(threadLocal, 'driver', driver)
    return driver

In [9]:
all_data = []

In [10]:
def get_data(driver, id_info):
    name, iso, url = id_info
    
    url = 'http://www.hoovers.com' + url
    driver.get(url)
    
    data = {}
    data['ctx_source'] = url
    data['hoovers_name'] = name
    data['hoovers_iso'] = iso
    
    elem = WebDriverWait(driver, 6).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "hero"))
    )
    
    try:
        data['address'] = driver.find_element_by_class_name('address').text
    except: pass
    
    try:
        data['type'] = driver.find_element_by_class_name('type').text
    except: pass
    
    try:
        data['role'] = driver.find_element_by_class_name('role').text
    except: pass
        
    try:
        data['revenue'] = driver.find_element_by_css_selector('li[data-vtab=vtab1]').text.split(' ')
        if len(data['revenue']) == 4:
            if data['revenue'][0] != '': data['revenue_year'] = int(data['revenue'][0])
            data['revenue'] = float(data['revenue'][3].replace('$', '').replace(',', '')) * 1000000
    except: pass
    
    try:
        data['gross_profit'] = driver.find_element_by_css_selector('li[data-vtab=vtab2]').text.split(' ')
        if len(data['gross_profit']) == 4:
            data['gross_profit'] = float(data['gross_profit'][3].replace('$', '').replace(',', '')) * 1000000
    except: pass
        
    try:
        data['operating_income'] = driver.find_element_by_css_selector('li[data-vtab=vtab3]').text.split(' ')
        if len(data['operating_income']) == 4:
            data['operating_income'] = float(data['operating_income'][3].replace('$', '').replace(',', '')) * 1000000
    except: pass
    
    try:
        data['net_income'] = driver.find_element_by_css_selector('li[data-vtab=vtab4]').text.split(' ')
        if len(data['net_income']) == 4:
            data['net_income'] = float(data['net_income'][3].replace('$', '').replace(',', '')) * 1000000
    except: pass
        
    try:
        data['assets'] = driver.find_element_by_class_name('incomeassets').text.split('\n')
        data['assets_units'] = data['assets'][0].replace('ASSETS ', '').replace('(', '').replace(')', '')
        data['assets'] = float(data['assets'][1].replace(',', ''))
    except: pass
    
    try:
        data['founded'] = driver.find_element_by_class_name('founded').text.split('\n')
        if '-' not in data['founded'][1]: data['founded'] = float(data['founded'][1].replace(',', ''))
        else:
            data['founded'] = float(data['founded'][1].split('-', '')[0])
    except: pass
    
    try:
        data['market_value'] = driver.find_element_by_class_name('marketval').text.split('\n')
        data['market_value_units'] = data['market_value'][0].replace('MARKET VALUE ', '').replace('(', '').replace(')', '')
        data['market_value'] = float(data['market_value'][1].replace(',', ''))
    except: pass
    
    try:
        data['sales_growth_percent'] = driver.find_element_by_class_name('salesgrowthpercent').text.split('\n')
        data['sales_growth_percent'] = float(data['sales_growth_percent'][1].replace('%', ''))
    except: pass
    
    try:
        data['employee_growth_percent'] = driver.find_element_by_class_name('empgrowthpercent').text.split('\n')
        data['employee_growth_percent'] = float(data['employee_growth_percent'][1].replace('%', ''))
    except: pass
    
    try:
        data['net_income_growth_percent'] = driver.find_element_by_class_name('netincomegrowthpercent').text.split('\n')
        data['net_income_growth_percent'] = float(data['net_income_growth_percent'][1].replace('%', ''))
    except: pass
        
    try:
        data['num_subsidiaries_parent'] = driver.find_element_by_css_selector('div[name=subsidiary]').text.split('\n')[0]
        data['num_subsidiaries_parent'] = int(data['num_subsidiaries'].replace(' Subsidiaries', ''))
    except: pass
    
    try:
        data['num_branches_parent'] = driver.find_element_by_css_selector('div[name=branch]').text.split('\n')[0]
        data['num_branches_parent'] = int(data['num_branches'].replace(' Branches', ''))
    except: pass
    
    try:
        data['parent'] = driver.find_element_by_css_selector('div[name=parent]').text.split('\n')[0]
    except: pass
    
    return data

In [11]:
from unidecode import unidecode

def load_page(row):
    _, row = row
    
    driver = get_driver()
    try:
        driver.get('http://www.hoovers.com/company-information/company-search.html?term=' + unidecode(row['name']))
        
        elem = WebDriverWait(driver, 8).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "cmp-company-directory"))
        )

        names = driver.find_elements_by_class_name('company_name')
        locations = driver.find_elements_by_class_name('company_location')
        profiles = driver.find_elements_by_class_name('company_profile')
        
    except:
        time.sleep(2)
        driver.get('http://www.hoovers.com/company-information/company-search.html?term=' + row['name'])
        
        elem = WebDriverWait(driver, 8).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "cmp-company-directory"))
        )
        
        names = driver.find_elements_by_class_name('company_name')
        locations = driver.find_elements_by_class_name('company_location')
        profiles = driver.find_elements_by_class_name('company_profile')
    
    data = {}
    try:
        companies = [(names[i].text,
                      locations[i].text,
                      profiles[i].get_attribute('innerHTML')) for i in range(len(names))]
        companies = [(n, l.split(', ')[-1], p.split('"')[1]) for n,l,p in companies]
        companies = [(n, standardize_iso(l, inp=False), p) for n,l,p in companies]
        comp = [c for c in companies if c[1] == row['iso']]

        if len(comp) >= 1: data = get_data(driver, comp[0])
        elif len(comp) == 0: data = {}
        else: data = get_data(driver, companies[0])
    except:
        pass
        
    data['name'] = row['name']
    data['iso'] = row['iso']
    data['entity_type'] = row['entity_type']
        
    all_data.append(data)
    if len(all_data) % 10 == 0: print(len(all_data), len(cdp), time.time())

ThreadPool(6).map(load_page, cdp.iterrows())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/Cellar/ipython/7.2.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-7f51106541f2>", line 52, in <module>
    ThreadPool(6).map(load_page, cdp.iterrows())
  File "/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 268, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 651, in get
    self.wait(timeout)
  File "/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 648, in wait
    self._event.wait(timeout)
  File "/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 552, in w

KeyboardInterrupt: 

In [12]:
ad = pd.DataFrame(all_data)
ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 25 columns):
address                      922 non-null object
assets                       683 non-null object
assets_units                 683 non-null object
ctx_source                   922 non-null object
employee_growth_percent      638 non-null object
entity_type                  1011 non-null object
founded                      538 non-null object
gross_profit                 603 non-null object
hoovers_iso                  922 non-null object
hoovers_name                 922 non-null object
iso                          998 non-null object
market_value                 673 non-null object
market_value_units           673 non-null object
name                         1011 non-null object
net_income                   778 non-null object
net_income_growth_percent    603 non-null object
num_branches_parent          840 non-null object
num_subsidiaries_parent      856 non-null object
operating

In [18]:
ad.to_csv('cdp_gapfilled.csv')

In [25]:
ad[ad['revenue'].apply(lambda x: type(x) == float and not pd.isna(x))].shape

(776, 25)